In [22]:
# initial setup
try:
    # settings colab:
    import google.colab
        
except ModuleNotFoundError:    
    # settings local:
    %run "../../../common/0_notebooks_base_setup.py"

pandas=1.0.3 already installed
matplotlib=2.2.2 already installed
bokeh=2.0.0 already installed
seaborn=0.10.0 already installed


In [23]:
import pandas as pd
import seaborn as sns
import re
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
import chart_studio
import chart_studio.plotly as py
import unidecode as ud

In [24]:
pd.set_option('display.max_rows', 300)
pd.set_option('display.width', 5000)

In [25]:
import chardet
def get_encoding_type(csv_path):
    rawdata = open(csv_path, 'rb').read()
    result = chardet.detect(rawdata)
    return result.get('encoding')

In [26]:
data = pd.read_csv("properati.csv", encoding="iso-8859-1")

In [27]:
#Creo una copia del DF
data_clean = data.copy(deep=True)

In [28]:
#Columnas que son string
columnas_str = ['operation', 'property_type', 'place_name', 'place_with_parent_names', 'country_name', 'state_name', 'description', 'title']

In [29]:
#Le quito los espacios en blanco
for columna in columnas_str:
    data_clean[columna] = data_clean[columna].str.strip()

<span style="font-size:24px">OPERACION<span>

In [30]:
#BUSCAR NULOS
data_clean["property_type"].isnull().sum()

0

In [31]:
#Expresión regular para buscar venta o alquiler
regex = re.compile("(venta)|(alquiler)", flags = re.IGNORECASE)

In [32]:
#Buscar Venta o Alquiler en description
regexOperacionesDescription = data_clean.description.apply(lambda x: regex.search(str(x)))
dfVentasDescription = regexOperacionesDescription.apply(lambda x: np.NaN if x is None else x.group(0))

#Normalizar
dfVentasDescription[dfVentasDescription.notnull()].unique()
dfVentasDescription.replace(["VENTA", "venta"], "Venta", inplace=True)
dfVentasDescription.replace(["alquiler", "ALQUILER"], "Alquiler", inplace=True)
dfVentasDescription[dfVentasDescription.notnull()].unique()

#Crear columna Operacion
data_clean["Operacion_Description"] = dfVentasDescription
data_clean["Operacion_Description"].value_counts()

Venta       4624
Alquiler     101
Name: Operacion_Description, dtype: int64

In [33]:
#Buscar Venta o Alquiler en title
regexOperacionesTitle = data_clean.title.apply(lambda x: regex.search(str(x)))
dfVentasTitle = regexOperacionesTitle.apply(lambda x: np.NaN if x is None else x.group(0))

#Normalizar
dfVentasTitle[dfVentasTitle.notnull()].unique()
dfVentasTitle.replace(["VENTA", "venta"], "Venta", inplace=True)
dfVentasTitle.replace(["alquiler", "ALQUILER"], "Alquiler", inplace=True)
dfVentasTitle[dfVentasTitle.notnull()].unique()

#Crear columna Title
data_clean["Operacion_Title"] = dfVentasTitle
data_clean["Operacion_Title"].value_counts()

Venta       6523
Alquiler      10
Name: Operacion_Title, dtype: int64

In [34]:
#Buscar Venta o Alquiler en properati_url
regexOperacionesURL = data_clean.properati_url.apply(lambda x: regex.search(str(x)))
dfVentasURL = regexOperacionesURL.apply(lambda x: np.NaN if x is None else x.group(0))

#Normalizar
dfVentasURL[dfVentasURL.notnull()].unique()
dfVentasURL.replace(["VENTA", "venta"], "Venta", inplace=True)
dfVentasURL.replace(["alquiler", "ALQUILER"], "Alquiler", inplace=True)
dfVentasURL[dfVentasURL.notnull()].unique()

#Crear columna URL
data_clean["Operacion_URL"] = dfVentasURL
data_clean["Operacion_URL"].value_counts()

Venta       12793
Alquiler       38
Name: Operacion_URL, dtype: int64

In [35]:
#Crear la columna Operacion con Operacion_URL que contempla todos los casos
data_clean["Operacion"] = data_clean["Operacion_URL"]

In [36]:
#Precios mínimos y máximos para alquiler
data_clean[data_clean.price_aprox_usd > 0].filter(items=["property_type", "price_aprox_usd"]).groupby(by="property_type").agg(['min', 'max'])

price_aprox_usd            
                          min         max
property_type                            
PH                   14020.91   2750000.0
apartment             5000.00   4000000.0
house                 7010.45  11000000.0
store                 8412.55   4600000.0

In [37]:
#Revisar los casos con precios menores a 10.000 para ver que no sean alquileres
data_clean[(data_clean.property_type == "house") & (data_clean.price_aprox_usd < 10000)].filter(items=["price_aprox_usd", "description"])

,price_aprox_usd,description
7440,7010.45,"EXCELENTE CABAÃA 3 DORM, 2 BAÃOS, PARQUE, A ..."
9665,9534.22,CODIGO: 7 ubicado en: Juana Manuela Gorritti -...
9761,0.00,Casa en Venta de 2 dorm. en Armenia
12372,8412.55,FRACCION DE TERRENO AL FONDOA SUBDIVIDIR. AL F...


In [38]:
#Revisar los casos con precios menores a 10.000 para ver que no sean alquileres
data_clean[(data_clean.property_type == "apartment") & (data_clean.price_aprox_usd < 10000)].filter(items=["price_aprox_usd", "description"])

,price_aprox_usd,description
4399,5000.00,Corredor Responsable: Mauro Marvisi - CMCPSI 5...
6720,5608.31,BV Rondeau / Nancen: A metros del Shoping Port...


In [39]:
#Revisar los casos con precios menores a 10.000 para ver que no sean alquileres
data_clean[(data_clean.property_type == "store") & (data_clean.price_aprox_usd < 10000)].filter(items=["price_aprox_usd", "description"])

,price_aprox_usd,description
836,8412.55,"Venta de Negocio en Tolosa, La Plata116 entre..."
